## image classification using cnn and tensorflow

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataset_path = os.listdir("./dataset/")
room_types = os.listdir("./dataset/")

print(f"types of room found: {len(room_types)}")

types of room found: 3


In [8]:
rooms = []
for item in room_types:
    all_room_types = os.listdir('./dataset/'+item)
    
    for room in all_room_types:
        rooms.append((item, str("./dataset/"+item)+"/"+room))
        
rooms[:5]

[('bed_room', './dataset/bed_room/bed-1303451__340.jpg'),
 ('bed_room', './dataset/bed_room/bed-1846251__340.jpg'),
 ('bed_room', './dataset/bed_room/bed-3786264__340.jpg'),
 ('bed_room', './dataset/bed_room/bed-4065946__340.jpg'),
 ('bed_room', './dataset/bed_room/bed-4343379__340.jpg')]

In [9]:
#build a dataframe
rooms_df = pd.DataFrame(data=rooms, columns=['room_type', "image"])
print(rooms_df.head())
print(rooms_df.tail())

  room_type                                    image
0  bed_room  ./dataset/bed_room/bed-1303451__340.jpg
1  bed_room  ./dataset/bed_room/bed-1846251__340.jpg
2  bed_room  ./dataset/bed_room/bed-3786264__340.jpg
3  bed_room  ./dataset/bed_room/bed-4065946__340.jpg
4  bed_room  ./dataset/bed_room/bed-4343379__340.jpg
      room_type                                           image
42  living_room  ./dataset/living_room/pexels-photo-245208.jpeg
43  living_room  ./dataset/living_room/pexels-photo-275484.jpeg
44  living_room  ./dataset/living_room/pexels-photo-276583.jpeg
45  living_room  ./dataset/living_room/pexels-photo-276724.jpeg
46  living_room  ./dataset/living_room/pexels-photo-279719.jpeg


In [10]:
print(f"total no. of rooms in the dataset: {len(rooms_df)}")

total no. of rooms in the dataset: 47


In [11]:
room_count = rooms_df["room_type"].value_counts()
print("rooms in each category: ")
print(room_count)

rooms in each category: 
living_room    24
bed_room       12
dining_room    11
Name: room_type, dtype: int64


In [12]:
import cv2
path = "./dataset/"
img_size = 60

images = []
labels = []

for i in room_types:
    data_path = path+str(i)
    filenames = [ i for i in os.listdir(data_path) ]
    for f in filenames:
        img = cv2.imread(data_path+"/"+f)
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(i)

In [13]:
#transform image array to a np type
images = np.array(images)
images.shape

(47, 60, 60, 3)

In [14]:
images = images.astype("float32")/255.0

In [15]:
images.shape

(47, 60, 60, 3)

In [23]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

y = rooms_df["room_type"].values

print(y[:5])

['bed_room' 'bed_room' 'bed_room' 'bed_room' 'bed_room']


In [24]:
#for y
y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2]


In [26]:
y = y.reshape(-1,1)
ct = ColumnTransformer([("room_type", OneHotEncoder(), [0])], remainder = 'passthrough')
Y = ct.fit_transform(y)
Y.shape

(47, 3)

In [33]:
#splitting dataset

In [32]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state=42)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.1, random_state=1)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(42, 60, 60, 3)
(42, 3)
(5, 60, 60, 3)
(5, 3)


In [38]:
#training 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

num_classes = 3

#flattened image
i_input = 10800

#hyperparameter
learning_rate = 0.001
trainin_iter = 10
batch_size = 16

Instructions for updating:
non-resource variables are not supported in the long term


In [39]:
img_size = 60
num_channels = 3

x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels]) #None, 60, 60, 3
y_ = tf.placeholder(tf.float32, shape=[None, num_classes])

print(f"shape of placeholder: {x.shape}, {y_.shape}")

shape of placeholder: (?, 60, 60, 3), (?, 3)


In [41]:
def conv2d(x, w, b, strides=1):
    x = tf.nn.conv2d(x, w, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1,k,k,1], padding="SAME")

In [42]:
weights = {
    'w1': tf.Variable(tf.random_normal([5, 5, 3, 32]),name='w1'),
    'w2': tf.Variable(tf.random_normal([5, 5, 32, 64]),name='w2'),
    'w3': tf.Variable(tf.random_normal([5, 5, 64, 128]),name='w3'),
    'wd1': tf.Variable(tf.random_normal([8 * 8 * 128, 2048]),name='wd1'),  
    'wout': tf.Variable(tf.random_normal([2048, num_classes]),name='wout')
}

biases = {
    'b1': tf.Variable(tf.random_normal([32]),name='b1'),
    'b2': tf.Variable(tf.random_normal([64]),name='b2'),
    'b3': tf.Variable(tf.random_normal([128]),name='b3'),
    'bd1': tf.Variable(tf.random_normal([2048]),name='bd1'),
    'bout': tf.Variable(tf.random_normal([num_classes]),name='bout')
}

In [43]:
def conv_net(x, weights, biases):
        
    # reshape input to 60x60x3 size
    x = tf.reshape(x, shape=[-1, 60, 60, 3])  
    
    print("###########################################################################")
    print("size of x is")
    print(x.shape)
    
    conv1 = conv2d(x, weights['w1'], biases['b1'])
    conv1 = maxpool2d(conv1, k=2)
    print("###########################################################################")
    print("size after 1st conv layer is ")
    print(conv1.shape)
    
    #input is 30*30*32 image
    # Convolution Layer
    conv2 = conv2d(conv1, weights['w2'], biases['b2'])
    conv2 = maxpool2d(conv2, k=2)
    print("###########################################################################")
    print("size after 2nd conv and pooling layer is")
    print(conv2.shape)
    
    ### third conv layer
    # input is 15*15*64 image
    # Convolution Layer
    conv3 = conv2d(conv2, weights['w3'], biases['b3'])
  
    conv3 = maxpool2d(conv3, k=2)
    print("###########################################################################")
    print("size after 3rd conv and pooling layer is")
    print(conv3.shape)
    
    #input is 8*8*128 

    # Fully connected layer
    # Reshape conv3 output to fit fully connected layer input   = 8*8*128 = 8192
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    print("###########################################################################")
    print("shape after flattening the image")
    print(fc1)  #8192 is the output
    
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    print("###########################################################################")
    print("shape after fully connected layer")
    print(fc1)
    
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['wout']), biases['bout'])
    print("###########################################################################")
    print("Output layer")
    return out

In [44]:
# Create the model
model = conv_net(x, weights, biases)
print(model)

###########################################################################
size of x is
(?, 60, 60, 3)
###########################################################################
size after 1st conv layer is 
(?, 30, 30, 32)
###########################################################################
size after 2nd conv and pooling layer is
(?, 15, 15, 64)
###########################################################################
size after 3rd conv and pooling layer is
(?, 8, 8, 128)
###########################################################################
shape after flattening the image
Tensor("Reshape_1:0", shape=(?, 8192), dtype=float32)
###########################################################################
shape after fully connected layer
Tensor("Relu_3:0", shape=(?, 2048), dtype=float32)
###########################################################################
Output layer
Tensor("Add_1:0", shape=(?, 3), dtype=float32)


In [45]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [46]:
# Initializing the variables
init = tf.global_variables_initializer()

In [50]:
cost_history=[]
n_epochs =10
# the execution
sess = tf.Session()
sess.run(init)

# train_X=train_X1.toarray()
# test_X=test_X1.toarray()
# train_y=train_y.toarray()
# train_y=train_y.todense()

for i in range(n_epochs):
    a, c = sess.run([optimizer, cost], feed_dict={x: train_x, y_: train_y})  #working
    cost_history = np.append(cost_history,c)  # working
    print('epoch : ', i,  ' - ', 'cost: ', c) #working 

epoch :  0  -  cost:  3623597.2
epoch :  1  -  cost:  4459202.5
epoch :  2  -  cost:  6155002.5
epoch :  3  -  cost:  5254596.0
epoch :  4  -  cost:  4831837.5
epoch :  5  -  cost:  3213131.8
epoch :  6  -  cost:  1263566.9
epoch :  7  -  cost:  963663.1
epoch :  8  -  cost:  2343466.8
epoch :  9  -  cost:  1503944.2


In [51]:
# test_y=test_y.todense()  #working solution of ValueError: setting an array element with a sequence.
#print(test_y)

correct_prediction = tf.equal(tf.argmax(model,1), tf.argmax(y_,1))   
correct_prediction 

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy


# retrun the accuracy on the test set.
print("Accuracy: ", sess.run(accuracy, feed_dict={x: test_x, y_:test_y}))

Accuracy:  0.6
